In [1]:
import torch
from transformers import pipeline
import nltk

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

import csv

In [4]:
# model_path = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
# sentiment_analysis_pipe = pipeline(
#     "sentiment-analysis", 
#     model=model_path, 
#     tokenizer=model_path, 
#     device=0 if torch.cuda.is_available() else -1)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
import csv

# Path to your CSV file
filename = "data/reviews.csv"

# Empty list to store the first column
first_column = []

# Open the file in read mode
with open(filename, 'r') as csvfile:
  # Create a CSV reader object
  csv_reader = csv.reader(csvfile)

  # Skip the header row (optional, adjust based on your CSV)
  next(csv_reader, None)

  # Iterate through each row
  for row in csv_reader:
    # Extract the first element (assuming comma delimiter)
    first_column.append(row[0])

In [3]:
first_column[:1]

['We visited the Citadel with great expectations. It is very large, with a good collection of black n white photos of their kings and society hundreds of years ago.\nWe were most disappointed to see a flat empty open space where the Forbidden City once stood - bombed and destroyed completely during the war. Much of the rest of the imperial complex had also been similarly destroyed. Restoration work had recreated some spaces but much more needs to be done.\nWith a good guide, we were able to appreciate the highlights of its history.']

In [4]:
nltk.download('punkt')  # Download sentence tokenizer if not already installed

def paragraphs_to_sentences(paragraphs):
  """
  This function takes a list of paragraphs (strings) and returns a list of sentences (strings).

  Args:
      paragraphs: A list of strings, where each string represents a paragraph.

  Returns:
      A list of strings, where each string represents a sentence.
  """
  sentences = []
  for paragraph in paragraphs:
    sentences.extend(nltk.sent_tokenize(paragraph))
  return sentences

sentences = paragraphs_to_sentences(first_column)

[nltk_data] Downloading package punkt to /home/khanh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
print(len(sentences))
sentences[0]

29648


'We visited the Citadel with great expectations.'

In [17]:
# sentiment_analysis_pipe("Covid cases are increasing fast!")

[{'label': 'negative', 'score': 0.723576545715332}]

In [15]:
# sentiment_analysis_pipe(sentences[:1])

[{'label': 'positive', 'score': 0.9739198088645935}]

In [14]:
# type(sentiment_analysis_pipe(sentences[:1])[0])

dict

In [7]:
texts = sentences

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
with open('output.csv', 'w', newline='') as csvfile:
   fieldnames = ['text', 'positive_score', 'negative_score', 'neutral_score']
   writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
   writer.writeheader()

   for text in texts:
       encoded_input = tokenizer(text, return_tensors='pt')
       output = model(**encoded_input)
       scores = output[0][0].detach().numpy()
       scores = softmax(scores)

       positive_score = scores[config.label2id['positive']]
       negative_score = scores[config.label2id['negative']]
       neutral_score = scores[config.label2id['neutral']]

       writer.writerow({
           'text': text,
           'positive_score': positive_score,
           'negative_score': negative_score,
           'neutral_score': neutral_score
       })

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
